# Sentinel metadata generator

    Author: Ben Ross
    Date: 27th April 2021
    
Purpose: 

In [129]:
import folium
# from sentinelsat.sentinel import SentinelAPI, read_geojson, geojson_to_wkt
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date
import pandas as pd
import geopandas as gpd

In [130]:
# brisbane = geojson_to_wkt(read_geojson(r'files\brisbane.geojson'))
mines = gpd.read_file(r'C:\Users\rossb1\OneDrive - ITP (Queensland Government)\gis\playground\playground.gdb', layer='coal_mines_map_series_shapes')
mines.to_crs('epsg:4326', inplace=True)

C:\Anaconda\envs\py37\lib\site-packages\geopandas\geodataframe.py:577: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


In [131]:
mines['Project_Name']

0               The Range
1            Wilkie Creek
2              New Acland
3    Wandoan Coal Project
4                Elimatta
5             Kogan Creek
6               Commodore
7            Cameby Downs
Name: Project_Name, dtype: object

In [169]:
wilkiecreek = mines.iloc[1]['geometry']
newacland = mines.iloc[2]['geometry']
kogancreek = mines.iloc[5]['geometry']
commodore = mines.iloc[6]['geometry']
cambeydowns = mines.iloc[7]['geometry']

To obtain Sentinel-1 data

In [143]:
products = api.query(footprint,
                     date = ('20200901', date(2020, 12, 31)),
                     producttype='SLC',
                     orbitdirection='ASCENDING')
# api.download_all(products)

products_gdf = api.to_geodataframe(products)

C:\Anaconda\envs\py37\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
C:\Anaconda\envs\py37\lib\site-packages\sentinelsat\sentinel.py:469: FutureWarning: Assigning CRS to a GeoDataFrame without a geometry column is now deprecated and will not be supported in the future.
  return gpd.GeoDataFrame(crs=crs)


To obtain Sentinel 2 data

In [178]:
# by default sentinelsat qill return results from the last 24 hours only

api = SentinelAPI('benross', 'password', 'https://scihub.copernicus.eu/apihub')
products = api.query(wilkiecreek, date = ('20180601', '20180630'), platformname = 'Sentinel-2', cloudcoverpercentage = (0, 30))

print('Products returned by search: {n}'.format(n=len(products)))
products_gdf = api.to_geodataframe(products)

Products returned by search: 19


C:\Anaconda\envs\py37\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [173]:
products_gdf['cloudcoverpercentage']

2a23e3b4-69ca-4376-9c58-48d7fe01e6cf     8.4146
ab07f641-0de5-4806-9177-69ba40ec4bdb    23.2839
2ce61795-ef11-4656-ab0e-61de650a7041    24.0044
0f7630f5-adc1-4ae7-b4e6-2bb6b6eaf120     0.0089
6586043c-a066-4ee0-9551-55e5d49d1f21     0.0000
7aa4e4e5-57a2-47b7-be34-693e02006de4     0.0000
74d52263-6200-4a94-962e-7c647b037c5e     0.0000
32c509dc-0bcd-480f-bc7b-3bc2f32aafcf     0.0000
d45b0d48-ee2e-49bd-b908-8849f5bfca3e     0.0000
863df6ef-aa8d-4ff4-933e-2459dab5b90c     0.2473
68610d9d-43ba-4736-899c-a7a9eb8401f2     0.0133
40455611-651e-404c-a536-50e38368a530    21.2956
312dcd0f-6d0c-460f-8850-3d1ae6627402    23.8729
387602c0-0c76-4999-b5a1-e75de94536ac     2.6960
9b7ef90d-c30a-4c22-bb15-095a9d84e18d     1.1195
488c0d92-90b4-4cf6-a421-84c3be20a623     0.0011
bd2a6d0b-e021-4691-8737-e2b79fc4ac2c     0.0000
8db420e0-6a4a-4411-acfd-4b5ba7905bbc     0.0000
bbd39370-164a-4fd6-a06b-5bc5469279bc     0.0000
Name: cloudcoverpercentage, dtype: float64

In [181]:
# save the search results as a csv if you like
# products_gdf.to_csv(r'files\sentinel_products_rolling_index.csv',header=True,index = True)
products_gdf.to_csv(r'files\sentinel_products_rolling_index.csv', header=True, index=True)

with open(r'files\sentinel_products_rolling_index.csv', 'a') as f:
    products_gdf.to_csv(f, header=False, index = True)
    
# products_gdf.to_file(r'files\sentinel_products_rolling_index.geojson', driver='GeoJSON')

In [175]:
m = folium.Map([-27.00, 151.00], zoom_start=8)
folium.GeoJson(products_gdf.iloc[4]['geometry']).add_to(m)
folium.GeoJson(mines.iloc[1]['geometry']).add_to(m) # simply pick an index from the list above to change the target mine
m